# Portfolio Optimization using Python 

In [1]:
# Libraries 
import pathlib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.optimize as sci_opt
from pprint import pprint # A print library!
from sklearn.preprocessing import StandardScaler

# Libraries to download data
from fake_useragent import UserAgent # :o
from data_downloader.data_client import PriceHistory 

# Set display options for Pandas
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)

## Getting the data

In [6]:
# Define symbols
symbols = ['AAPL','KO','SBUX','AXP'] # My stocks at 6/9/2024 :)
number_of_symbols = len(symbols)

# If we don't have data grab it from the NASDAQ
if not pathlib.Path('data/stock_data.csv').exists():

    # Initialize the PriceHistory Client.
    price_history_client = PriceHistory(symbols=symbols,
                                        user_agent=UserAgent().chrome,
                                        
                                        )

    # Grab the data and dump it to a CSV file
    price_history_client.price_data_frame.to_csv('data/stock_data.csv',
                                                 index=False
                                                 )
    display(price_history_client.price_data_frame)

    # Grab the data frame
    price_data_frame : pd.DataFrame = price_history_client.price_data_frame
else:
    # Load the existing csv file.
    price_data_frame : pd.DataFrame = pd.read_csv('data/stock_data.csv')

print(price_data_frame.head())

         date   close   volume    open      high       low symbol
0  2024-06-07  232.67  2212477  232.41  234.1205  231.7100    AXP
1  2024-06-06  233.35  2709843  234.79  236.4000  232.7000    AXP
2  2024-06-05  234.69  2925098  236.25  237.7000  233.7000    AXP
3  2024-06-04  237.25  2512117  236.39  240.0000  235.7300    AXP
4  2024-06-03  236.88  2677196  240.18  241.3600  233.5309    AXP


In [7]:
# We want one column for each stock, so we need to reorganize the dataframe.
price_data_frame = price_data_frame[['date','symbol','close']]

# Symbols are headers
price_data_frame = price_data_frame.pivot(index='date',
                                          columns='symbol',
                                          values='close'
                                          )
display(price_data_frame)

symbol,AAPL,AXP,KO,SBUX
date,,,,
2023-12-11,193.18,168.39,59.04,98.12
2023-12-12,194.71,171.04,59.42,98.15
2023-12-13,197.96,174.56,59.93,98.11
2023-12-14,198.11,178.86,59.04,97.84
2023-12-15,197.57,180.51,58.60,96.75
...,...,...,...,...
2024-06-03,194.03,236.88,62.93,82.08
2024-06-04,194.35,237.25,63.94,82.79
2024-06-05,195.87,234.69,63.92,81.16


## Calculating Sharpe Ratio

We're using the portfolio of stocks to generate returns. To generate returns, we need to take on **risk**, but ideally we should be comfortable taking on this risk because we are compensated in the form returns that increase our investment.

Most investors aren't okay with taking on high levels of risk. Out goal is to find the best ratio risk vs. returns. Ideally, we are taking the most risk we are comfortable with and attempting to maximize those returns. 

***How do we measure risk?***

There are many ways to measure risk, and some are more appropiate than others depending on the situation at hand. In our case, we'll use a very popular metric, the _Sharpe Ratio_. The Sharpe Ratio is a measure for calculating risk-adjusted return and has been the industry standard for such caculations. It allows us to quentify the relationship between the average return earned in excess of the risk-free rate per unit of volatility or total risk. 

Mathematically, we define the Sharpe Ratio as the following: 

$$ \text{Sharpe Ratio} = \frac{R_p-R_f}{\sigma_p},$$

where:
- $R_p$: Return of Portfolio
- $R_f$: Risk-free rate
- $\sigma_p$: Standard-Deviation of Portfolio's Excess returns.
  
  To calculate the expected returns, we use the following formula:
  $$R_p = \sum_i^S w_ir_i,$$

  where
  - $r_i$: Return of security $i$
  - $w_i$: Weight of security $i$
  
  To calculate the standard deviation of the portfolio, we use the following formula:
  $$\sigma_p = \sqrt{\sum_i^S w_i^2\sigma_i^2 +\sum_{i,j}^S 2w_iw_jp_{ij}\sigma_i\sigma_j},$$
where
- $p_{ij}$: Correlation coefficient between the return of assets $i,j$.

